# Universidade Federal do Rio Grande do Norte
## Programa de Pós-graduação em Engenharia Elétrica e de Computação
### Course: Tópicos Especiais C - Data Science Foundation
### Professor: Ivanovitch Silva
### Students: 

* Aguinaldo Bezerra Batista Júnior
* Pedro Klisley Ferreira da Silva
* Ycaro Ravel Dantas

#### Activity: Project 4
#### Subject: IES employees gender analysis
#### Objectives: Analyze gender in IES employees from webscrapped data of Portal da Transparência 
#### Base dataset: number_of_employees.csv

### 0 Loading Libraries and base dataset

In [11]:
# Loading needed libraries
import pandas as pd
import requests as requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint

This activity starts with loading a base dataset obtained in professor's guiding notebook. This dataset was built on webscrapped data from [Portal da Transparência](http://www.portaldatransparencia.gov.br/).

In [14]:
# Loading data retrieved in the professor's guiding notebook
ies_df = pd.read_csv('number_of_employees.csv')
ies_df.head()

,Unnamed: 0,Code,IES_name,Number_employees
0,0,26257,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,1841
1,1,26256,CENTRO FED.EDUC.TECNOL.CELSO S. FONSECA,1572
2,2,26201,COLEGIO PEDRO II,2345
3,3,26443,EMPRESA BRAS. DE SERVICOS HOSPITALARES,25750
4,4,26298,FUND COORD APERF PESSOAL NIVEL SUPERIOR,409


In this initial webscrapping process, data was obtained from the [Ministry of Education](http://www.portaldatransparencia.gov.br/servidores//OrgaoExercicio-ListaOrgaos.asp?CodOS=15000) page in [Portal da Transparência](http://www.portaldatransparencia.gov.br/). In this activity we have to go deeper in the webscrapping proccess and obtain more data from several pages in that website.

### 1 Retrieving public employee data
The webscrapping process starts with the analysis of [Portal da Transparência](http://www.portaldatransparencia.gov.br/) webpage structure:
* Ministry of Education
    - IES employees pages

Each IES may have several pages listing employees, so we must access all of them to get the desired data

In [16]:
# Base URL to access the institutions's list of employees
base_url = 'http://www.portaltransparencia.gov.br/servidores/OrgaoExercicio-ListaServidores.asp?CodOrg={}'

In [18]:
ies_names = []
# Base page lookup
for ies in ies_df.Code:
    url = base_url.format(ies).replace(" ", "")
    
    # Retrieving content for the main page for the institution
    response = requests.get(url)
    
    # Pause the loop
    sleep(randint(5,10))
    
    # Parsing content
    ies_html = BeautifulSoup(response.text, 'html.parser')
    
    # Discovering the number of pages per institution
    npages = ies_html.find('p', class_='paginaAtual').text.rsplit('/',1)[1]
    
    ies_name = ies_html.find_all('td',class_='colunaValor')[1].text.strip()    
    
    # Gathering data from IES employee pages
    for page in range(1,int(npages)):
        
        page_url = url+'&Pagina={}'.format(page).replace(" ", "")
        
        resp = requests.get(page_url)
        
         # Pause the loop
        sleep(randint(5,10))
        
        # Retrieving content for the page
        page_html = BeautifulSoup(response.text, 'html.parser')
        
        page_table_rows = page_html.find_all('td')
        
    
        names = []
        # Getting names in each page
        for row in page_table_rows:
            name = row.find('a')
            
            if name is not None:
                names.append(name.text.strip())
                ies_names.append(ies_name)
                
    
        break
    
    
    break
    


Now, with scrapped data in hands, a new dataframe can be built. The absent gender information problem will be addressed in next section

In [23]:
# Building a new dataset based on the lists built from webscrapying
columns = ['Name','IES','Guessed_Gender']
ies_employees_df = pd.DataFrame(columns=columns)
ies_employees_df.Name = names
ies_employees_df.IES = ies_names
ies_employees_df.Guessed_Sex = None

In [24]:
ies_employees_df

,Name,IES,Guessed_Gender
0,ABELARDO BENTO ARAUJO,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,NaN
1,ABNER BRAGANCA GOUVEA,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,NaN
2,ACCACIO FERREIRA DOS SANTOS NETO,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,NaN
3,ADALBERTO JOSE DE LIMA,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,NaN
4,ADALBERTO PEREIRA DA SILVA,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,NaN
5,ADALBERTO TEIXEIRA DE ANDRADE ROCHA,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,NaN
6,ADALTON MARTINS GOMES,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,NaN
7,ADELSON DE PAULA SILVA,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,NaN
8,ADELSON FERNANDES MOREIRA,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,NaN
9,ADERCI DE FREITAS FILHO,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,NaN


### 2 Guessing Genders
Generally, person's first name is closed related to gender. So, it may be possible to guess gender from a person's firts name using some libraries and APIs.

Our first approach is to use the [GenderBR](http://fmeireles.com/blog/rstats/genderbr-predizer-sexo) package availabe in R (CRAN) to try to predict gender from a person's first name. We have chosen this package because it bases its prediction on IBGE official data. So, clearly it is an accurate and offline approach to guess gender for brazilian names.

In [31]:
# We are able to run R commands in ipython using this python library
import rpy2.robjects as robjects
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [32]:
%%R 
# Need to load this R library for gender guessing
suppressMessages(library(genderBR))


Error: package or namespace load failed for ‘genderBR’


/opt/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Error in dyn.load(file, DLLpath = DLLpath, ...) : 
  unable to load shared object '/opt/anaconda3/lib/R/library/rlang/libs/rlang.so':
  /opt/anaconda3/lib/R/library/rlang/libs/rlang.so: undefined symbol: R_ExternalPtrAddrFn

  warnings.warn(x, RRuntimeWarning)
/opt/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Error: package or namespace load failed for ‘genderBR’

  warnings.warn(x, RRuntimeWarning)


In [30]:
genders = []
# Using robjects to run R procedures
for name in ies_employees_df.Name:
        name = name.split()[0]
        gender = robjects.r.get_gender(name, threshold = 0.95)
        gender = gender.r_repr()
        gender = gender.strip('\"')
        if gender.startswith('NA'):
            gender = None
        genders.append(gender)

AttributeError: 'R' object has no attribute 'get_gender'

In [120]:
ies_employees_df.Guessed_Gender = genders

In [121]:
ies_employees_df

,Name,IES,Guessed_Gender
0,ABELARDO BENTO ARAUJO,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,Male
1,ABNER BRAGANCA GOUVEA,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,Male
2,ACCACIO FERREIRA DOS SANTOS NETO,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,Male
3,ADALBERTO JOSE DE LIMA,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,Male
4,ADALBERTO PEREIRA DA SILVA,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,Male
5,ADALBERTO TEIXEIRA DE ANDRADE ROCHA,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,Male
6,ADALTON MARTINS GOMES,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,Male
7,ADELSON DE PAULA SILVA,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,Male
8,ADELSON FERNANDES MOREIRA,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,Male
9,ADERCI DE FREITAS FILHO,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,None


R's GenderBR fails to detect genders associated to some first names. So we can submit these cases to the genderize_io API.

In [165]:
# No API key required for genderize.io
def genderize_io(genderless_name):
    genderless_name = genderless_name.split()[0]
    api_url='https://api.genderize.io/?name={}'.format(genderless_name).replace(" ", "")
    gender = requests.get(api_url,timeout=10).json().get('gender')
    if(gender):
        return gender.capitalize()
    else:
        return None

In [163]:
for index,row in ies_employees_df.iterrows():
    if row[2] is None:
        gen = genderize_io(row[1])
        if(gen):
            ies_employees_df.Guessed_Gender.loc[index] = gen
        
        

In [164]:
ies_employees_df

,Name,IES,Guessed_Gender
0,ABELARDO BENTO ARAUJO,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,Male
1,ABNER BRAGANCA GOUVEA,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,Male
2,ACCACIO FERREIRA DOS SANTOS NETO,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,Male
3,ADALBERTO JOSE DE LIMA,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,Male
4,ADALBERTO PEREIRA DA SILVA,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,Male
5,ADALBERTO TEIXEIRA DE ANDRADE ROCHA,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,Male
6,ADALTON MARTINS GOMES,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,Male
7,ADELSON DE PAULA SILVA,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,Male
8,ADELSON FERNANDES MOREIRA,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,Male
9,ADERCI DE FREITAS FILHO,CENTRO FED.DE EDUC.TECNOL.MINAS GERAIS,Male
